In [ ]:
## This is the code to do the data analysis for Ale Mineo's cellpose cell detection.
# December 3, 2025
#
#
##Steps
#1.) Enter the data paths
#2.) Scan the data path for the folders that exist there
#3.) For each folder, bring in the 3 different size of cellpose cell detections maps, i.e. for each original image, there are three cellpose segmentation maps
#4.) Using region props, measure the volume for each
#5.) Scale if needed
#6.) Export to csv

In [1]:
import sys
import numpy as np
import pandas as pd
import os

import matplotlib
#matplotlib.rcParams["image.interpolation"] = 200
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tqdm import tqdm
from tifffile import imread

from skimage.segmentation import clear_border
from skimage.measure import label, regionprops, regionprops_table

import traceback


In [2]:
#Step 1, get the data
file_path = "/camp/home/fallest/Desktop/working/fallest/Projects/Ale_Mineo/Copy_Of_Datasets/Masked_Images/"
save_path = "/camp/home/fallest/Desktop/working/fallest/Projects/Ale_Mineo/Copy_Of_Datasets/Data_Out_Clear_Border/"

In [3]:
# Step 2 Scan the data path for the folders that exist there
def find_segmentation_tifs(root_folder):
    segmentation_files = []
    for dirpath, _, filenames in os.walk(root_folder):
        for fname in filenames:
            if "segmentation" in fname.lower() and fname.lower().endswith(".tif"):
                full_path = os.path.join(dirpath, fname)
                segmentation_files.append(full_path)
    return segmentation_files

In [4]:
segmentation_files = find_segmentation_tifs(file_path)  #scan in the files that are segmentation files

In [5]:
segmentation_files

['/camp/home/fallest/Desktop/working/fallest/Projects/Ale_Mineo/Copy_Of_Datasets/Masked_Images/masked_images_Mated/gdcomtrolmated_lif___TileScan_1_R_6_Merged_1_3_masked/gdcomtrolmated_lif___TileScan_1_R_6_Merged_1_3_masked.tifgdcomtrolmated_lif___TileScan_1_R_6_Merged_1_3_masked_10_pixels_segmentation.tif',
 '/camp/home/fallest/Desktop/working/fallest/Projects/Ale_Mineo/Copy_Of_Datasets/Masked_Images/masked_images_Mated/gdcomtrolmated_lif___TileScan_1_R_6_Merged_1_3_masked/gdcomtrolmated_lif___TileScan_1_R_6_Merged_1_3_masked.tifgdcomtrolmated_lif___TileScan_1_R_6_Merged_1_3_masked_20_pixels_segmentation.tif',
 '/camp/home/fallest/Desktop/working/fallest/Projects/Ale_Mineo/Copy_Of_Datasets/Masked_Images/masked_images_Mated/gdcomtrolmated_lif___TileScan_1_R_6_Merged_1_3_masked/gdcomtrolmated_lif___TileScan_1_R_6_Merged_1_3_masked.tifgdcomtrolmated_lif___TileScan_1_R_6_Merged_1_3_masked_15_pixels_segmentation.tif',
 '/camp/home/fallest/Desktop/working/fallest/Projects/Ale_Mineo/Copy_Of_D

In [6]:
def seg_clean_up(segmentations):
    #This is to be run on each segmentation file so it matches for region props, also checks that were segmentations
    segmentations[segmentations == -1] = 0  #converts -1 (unknown label) to zero
    segmentations = segmentations.astype(int) #converts to int
    return segmentations

In [7]:
def dict_to_df(data_dict):
    return pd.DataFrame({
        'label': data_dict['label'],
        'area': data_dict['area'],
        'x': data_dict['centroid-0'],
        'y': data_dict['centroid-1'],
        'z': data_dict['centroid-2'],
    })


In [8]:
for file in segmentation_files:
    try:
        file_name = os.path.splitext(os.path.basename(file))[0]
        file_name = file_name.partition('.tif')[2] #shortens the filename, as they are all weird.  Shold really be a seperate function
        file_full_path = os.path.dirname(file)
        image_dir = os.path.basename(os.path.dirname(file))
        print(f"Processing: {file_name}")

        segmentations = imread(file)
        segmentations = seg_clean_up(segmentations)
        segmentations = clear_border(segmentations)  #run if you want to clear the borders
        data_dict = regionprops_table(segmentations, properties=('label', 'area', 'centroid'))
        data_table = dict_to_df(data_dict)
        data_table_path = os.path.join(save_path, image_dir)

        if not os.path.exists(data_table_path):
            os.makedirs(data_table_path)

        output_csv = os.path.join(data_table_path, f"{file_name}clear_b.csv") #changed for clear border
        data_table.to_csv(output_csv)

        print(f" - saved: {output_csv}\n")

    except Exception as e:
        print(f"\n Error processing file: {file}")
        print(f"   {e}")
        traceback.print_exc()  # prints detailed stack trace
        print("   Skipping to next file...\n")


Processing: gdcomtrolmated_lif___TileScan_1_R_6_Merged_1_3_masked_10_pixels_segmentation
 - saved: /camp/home/fallest/Desktop/working/fallest/Projects/Ale_Mineo/Copy_Of_Datasets/Data_Out_Clear_Border/gdcomtrolmated_lif___TileScan_1_R_6_Merged_1_3_masked/gdcomtrolmated_lif___TileScan_1_R_6_Merged_1_3_masked_10_pixels_segmentationclear_b.csv

Processing: gdcomtrolmated_lif___TileScan_1_R_6_Merged_1_3_masked_20_pixels_segmentation
 - saved: /camp/home/fallest/Desktop/working/fallest/Projects/Ale_Mineo/Copy_Of_Datasets/Data_Out_Clear_Border/gdcomtrolmated_lif___TileScan_1_R_6_Merged_1_3_masked/gdcomtrolmated_lif___TileScan_1_R_6_Merged_1_3_masked_20_pixels_segmentationclear_b.csv

Processing: gdcomtrolmated_lif___TileScan_1_R_6_Merged_1_3_masked_15_pixels_segmentation
 - saved: /camp/home/fallest/Desktop/working/fallest/Projects/Ale_Mineo/Copy_Of_Datasets/Data_Out_Clear_Border/gdcomtrolmated_lif___TileScan_1_R_6_Merged_1_3_masked/gdcomtrolmated_lif___TileScan_1_R_6_Merged_1_3_masked_15_pix

In [9]:
file_name

'gceQF6reduced_lif___TileScan_1_A1_R_15_Merged_1_masked_15_pixels_segmentation'

In [10]:
file_name.partition('.tif')[2]

''